In [1]:
import numpy as np
import pandas as pd
import requests

In [2]:
r = requests.get('https://unitru.edu.pe/webfiles///Convocatoria/2021/10//35_DOC_CONVO_30102021063012.txt')
apparent_encoding = r.apparent_encoding
df = pd.read_csv('https://unitru.edu.pe/webfiles///Convocatoria/2021/10//35_DOC_CONVO_30102021063012.txt',
                 encoding=apparent_encoding,
                 names= { 'raw_data': 0})
df

,raw_data
0,...
1,OF.DE TECNOLOG.DE LA INFOR. EXA...
2,* AREA P.A.D.* ...
3,R...
4,=============================================...
...,...
1376,0010 037266 VELA PONCE DANIEL FABIO ...
1377,0011 039318 SICCHA CARRANZA JENIFFER LIZBETH ...
1378,0012 018972 DAVILA BENAVIDES MARINA LISBET ...
1379,********************************************...


In [3]:
type_test_and_place = ( df
 .iloc[1]
 .str.replace(r'\s+', ' ', regex=True)
 .str.split(r'EXAMEN\s\DE\sADMISION\s|\sP', expand=True)
 .loc['raw_data', 1]
 )
type_test_and_place

'ORDINARIO 2022- I - TRUJILLO'

In [4]:
date = (df
 .iloc[2]
 .str.extract(r'(\d+/\d+/\d+)')
 .loc['raw_data', 0]
 )
date

'30/10/2021'

In [7]:
row_area = (df
 .iloc[2]
 .str.replace(r'\s+', ' ', regex=True)
 ['raw_data']
 )
row_area

' * AREA P.A.D.* AREA : C-D 30/10/2021'

In [9]:
filter_rows = (df
               ['raw_data']
               .map(lambda row: row.startswith(' 0'))
               )
without_extra_spaces_S = (df
 .loc[filter_rows]
 .reset_index()
 .drop('index', axis=1)
 ['raw_data']
 .str.replace(r'\s+', ' ', regex=True)
 .str.strip()
 )

In [10]:
# without_extra_spaces_S.loc[0]

In [12]:
full_name_df = (without_extra_spaces_S
 .str.extract(r'(\s\D+\s-?\d)')
 [0]
 .str.replace(r'-?\d', '', regex=True)
 # .loc[195]
 # .str.contains(r'\d')
 # .value_counts()
 )

In [13]:
grades_df = ( without_extra_spaces_S
 .str.extractall(r'(-?\d+\.\d+)')
 .unstack()
 #.loc[191]
 )

In [16]:
grades_df.columns = grades_df.columns.droplevel(0) # solo se ejecuta una vez

In [17]:
grades_df.columns = grades_df.columns.rename(name=None) # solo se ejecuta una vez

In [18]:
# grades_df

In [19]:
# without_extra_spaces_S.loc[0]

In [20]:
school_with_detail_S = ( without_extra_spaces_S.str.extract(r'(\.\d+\s[A-Z].*)')
 #.loc[1178]
  [0]
  .str.replace(r'\.\d+\s', '', regex=True)
  # .value_counts()
 )

In [21]:
details_S = (school_with_detail_S
 .str.extract(r'(INGRESA.*|NO\sINGRESA.*|ING\..*|AUSENTE)')
 [0]
 )

In [22]:
school_S = ( school_with_detail_S
 .str.replace(r'INGRESA.*|NO\sINGRESA.*|ING\..*|AUSENTE', '', regex=True)
)

In [23]:
final_df = pd.concat(
    [full_name_df, grades_df, school_S, details_S],
    axis='columns',
    sort=False)

In [24]:
final_df.columns = ['full_name', 'apt_academica', 'conocimiento', 'total', 'minimo', 'escuela', 'observacion']
final_df.tail()

,full_name,apt_academica,conocimiento,total,minimo,escuela,observacion
1174,MOZO ZAVALETA JOSE LUIS,25.421,37.697,63.118,74.776,ED.SEC: HISTORIA,NO INGRESA
1175,PACHECO DEL AGUILA EDU,16.262,44.854,61.116,74.776,ED.SEC: HISTORIA,NO INGRESA
1176,VELA PONCE DANIEL FABIO,12.204,28.538,40.742,74.776,ED.SEC: HISTORIA,NO INGRESA
1177,SICCHA CARRANZA JENIFFER LIZBETH,19.325,15.275,34.600,74.776,ED.SEC: HISTORIA,NO INGRESA
1178,DAVILA BENAVIDES MARINA LISBET,12.204,-11.236,0.968,74.776,ED.SEC: HISTORIA,NO INGRESA


In [25]:
final_df['date'] = date
final_df['row_area'] = row_area
final_df['type_test_and_place'] = type_test_and_place

In [26]:
final_df = ( final_df
            .applymap(str.strip)
)

/tmp/ipykernel_1608/3200173129.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(str.strip)


In [27]:
pd.__version__

'2.1.1'

In [28]:
np.__version__

'1.25.2'